In [1]:
import numpy as np
import pandas as pd
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight, BM25Recommender

/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiment with the given example dataset

In [2]:
artists, users, artist_user_plays = get_lastfm()

In [3]:
type(artist_user_plays)

scipy.sparse._csr.csr_matrix

In [3]:
# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
# https://benfred.github.io/implicit/tutorial_lastfm.html#:~:text=The%20first%20step,classic%20information%20retrieval%3A
# https://en.wikipedia.org/wiki/Okapi_BM25
artist_user_plays = bm25_weight(artist_user_plays, K1=100, B=0.8)

# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_plays = artist_user_plays.T.tocsr()

In [4]:
model = BM25Recommender(K=20, K1=1.2, B=0.75, num_threads=0)
model.fit(user_plays)

/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.09476304054260254 seconds
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 292385/292385 [00:02<00:00, 111778.26it/s]


In [5]:
# Get recommendations for the a single user
userid = 12345
ids, scores = model.recommend(userid, user_plays[userid], N=10, filter_already_liked_items=False)
pd.DataFrame({"artist": artists[ids], "score": scores, "already_liked": np.in1d(ids, user_plays[userid].indices)})

,artist,score,already_liked
0,von thronstahl,8.183808e+07,True
1,triarii,6.715735e+07,True
2,rome,5.925941e+07,True
3,death in june,5.503927e+07,True
4,der blutharsch,5.243071e+07,False
5,dernière volonté,4.646192e+07,False
6,ordo rosarius equilibrio,4.608011e+07,False
7,arditi,3.672969e+07,True
8,a challenge of honour,3.627942e+07,True
9,the coffinshakers,3.618816e+07,True


In [6]:
# get related items for the beatles (itemid = 25512)
ids, scores= model.similar_items(252512)

# display the results using pandas for nicer formatting
pd.DataFrame({"artist": artists[ids], "score": scores})

,artist,score
0,the beatles,421510.447159
1,the rolling stones,173941.303199
2,john lennon,172970.060309
3,bob dylan,168919.391739
4,led zeppelin,157626.645538
5,the who,153671.149869
6,pink floyd,144742.286798
7,the beach boys,140239.720056
8,david bowie,137654.365234
9,the doors,133340.501471


In [ ]:
# Make recommendations for the first 1000 users in the dataset
userids = np.arange(1000)
ids, scores = model.recommend(userids, user_plays[userids])
ids, ids.shape

## Experiment with amazon beauty dataset

In [7]:
import numpy as np
import pandas as pd
from implicit.nearest_neighbours import bm25_weight, BM25Recommender
from utils import pandas_df_to_csr

In [10]:
amazon_beauty_df = pd.read_csv("ratings_Beauty.csv")

In [11]:
user_map, item_map, amazon_beauty_csr = pandas_df_to_csr(amazon_beauty_df)

In [12]:
amazon_beauty_csr_bm25 = bm25_weight(amazon_beauty_csr, K1=100, B=0.8)

In [13]:
amazon_beauty_csr_bm25 = amazon_beauty_csr_bm25.tocsr()

In [14]:
model = BM25Recommender(K=20, K1=1.2, B=0.75, num_threads=0)
model.fit(amazon_beauty_csr_bm25)

/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.013097047805786133 seconds
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 249274/249274 [00:01<00:00, 220947.84it/s]


In [15]:
# Get recommendations for the a single user
userid = 725046
ids, scores = model.recommend(userid, amazon_beauty_csr_bm25[userid], N=10, filter_already_liked_items=False)

In [16]:
pd.DataFrame({"ProductId": item_map.loc[ids]["ProductId"], "score": scores, "already_purchased": np.in1d(ids, amazon_beauty_csr_bm25[userid].indices)})

,ProductId,score,already_purchased
ItemIndex,,,
81854,B002OVV7F0,542641.439737,True
154092,B006GQPZ8E,365188.898138,True
89013,B0031IH5FQ,60292.014260,True
0,0205616461,41290.409158,True
154097,B006GQTZ8A,24972.735576,False
154084,B006GQG9GQ,16782.260942,False
156214,B006OZF74O,12100.925288,False
154076,B006GQB6BE,11318.057718,False
154078,B006GQDM0M,9403.656638,False


In [13]:
item_map.loc[ids]["ProductId"]

ItemIndex
114276    B0049WJA9C
65766     B001MA0QY2
181492    B008O4YM4Y
10253     B0009PVV40
103349    B003S516XO
174368    B0085WHBHU
75617     B002B9DWBC
89252     B00325D0WK
73976     B0027A7CLG
181069    B008MP481M
Name: ProductId, dtype: object

## End of Experiments

In [17]:
import numpy as np
import pandas as pd
import gc
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight, BM25Recommender
from implicit import evaluation
from utils import pandas_df_to_csr

## Pre-processing

In [18]:
amazon_beauty_df = pd.read_csv("ratings_Beauty.csv")

In [19]:
# Convert pandas df to CSR format
# user_map and item_map contains index in csr format to original id mappings for users and items repectively
user_map, item_map, amazon_beauty_csr = pandas_df_to_csr(amazon_beauty_df)

In [20]:
# weight the matrix, both to reduce impact of users that have puchased the same item thousands of times
# and to reduce the weight given to popular items
# Output is a COO matrix
amazon_beauty_coo_bm25 = bm25_weight(amazon_beauty_csr, K1=100, B=0.8)

### Train-test split

In [21]:
train_coo, test_coo = evaluation.train_test_split(amazon_beauty_coo_bm25, train_percentage=0.8, random_state=55)
print(f"Train size: {train_coo.size} \n Test size: {test_coo.size}")

Train size: 1618938 
 Test size: 404132


In [22]:
#Convert coo to csr
train_csr = train_coo.tocsr()
test_csr = test_coo.tocsr()

## Training

In [23]:
model = BM25Recommender(K=20, K1=1.2, B=0.75, num_threads=0)
model.fit(train_csr)

/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.010312080383300781 seconds
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 249274/249274 [00:01<00:00, 245746.33it/s]


## Evaluation

In [24]:
ranking_metrics_at_10 = evaluation.ranking_metrics_at_k(model, train_csr, test_csr, K=10, show_progress=True, num_threads=0)
ranking_metrics_at_10

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:09<00:00, 33719.68it/s]


{'precision': 0.031727250761608455,
 'map': 0.016171278581340893,
 'ndcg': 0.019896584816351613,
 'auc': 0.5135849715915827}

### Save model

In [29]:
model.save("6_BM25")

In [27]:
#Load model
loaded_model = BM25Recommender.load("6_BM25")

## Hyper-parameter Tuning

In [30]:
# Hyper-parameters lists
K = [10, 20, 30]
K1 = [0.5, 1.0, 1.5]
B = [0.5, 0.75, 1.0]

In [31]:
results = []
for k in K:
    for k1 in K1:
        for b in B:
            print("Training model with below parameter values --------")
            print(f"K: {k}, K1: {k1}, B: {b}")
            model =  BM25Recommender(K=k, K1=k1, B=b, num_threads=0)
            model.fit(train_csr)
            ranking_metrics_at_10 = evaluation.ranking_metrics_at_k(model, train_csr, test_csr, K=10, show_progress=True, num_threads=0)
            print("Evaluation results: \n", ranking_metrics_at_10)
            results.append(
                (k,k1,b,ranking_metrics_at_10['precision'],ranking_metrics_at_10['map'],ranking_metrics_at_10['ndcg'],ranking_metrics_at_10['auc'])
            )
            # Garbage handling
            del model
            gc.collect()

/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.011523008346557617 seconds
  warnings.warn(


Training model with below parameter values --------
K: 10, K1: 0.5, B: 0.5


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:09<00:00, 33724.12it/s]
/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.009142875671386719 seconds
  warnings.warn(


Evaluation results: 
 {'precision': 0.03656218785262534, 'map': 0.0182191915497098, 'ndcg': 0.022569413271458085, 'auc': 0.5156008956475355}
Training model with below parameter values --------
K: 10, K1: 0.5, B: 0.75


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:09<00:00, 33399.73it/s]
/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.008931159973144531 seconds
  warnings.warn(


...

Evaluation results: 
 {'precision': 0.033117201764590126, 'map': 0.01673763545629646, 'ndcg': 0.020638310949033224, 'auc': 0.5141505048576986}
Training model with below parameter values --------
K: 30, K1: 1.5, B: 0.75


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:10<00:00, 33075.73it/s]
/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.008889198303222656 seconds
  warnings.warn(


Evaluation results: 
 {'precision': 0.03085292674360386, 'map': 0.015646599975312103, 'ndcg': 0.019268941626350174, 'auc': 0.513177063613129}
Training model with below parameter values --------
K: 30, K1: 1.5, B: 1.0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:10<00:00, 32993.04it/s]

Evaluation results: 
 {'precision': 0.028758036432658368, 'map': 0.014687924939606887, 'ndcg': 0.018065046823911413, 'auc': 0.5123471914271797}


In [32]:
results_df = pd.DataFrame(results, columns=['K', 'K1', 'B', 'precision', 'map', 'ndcg', 'auc'])
results_df

,K,K1,B,precision,map,ndcg,auc
0,10,0.5,0.50,0.036562,0.018219,0.022569,0.515601
1,10,0.5,0.75,0.035304,0.017778,0.021949,0.515061
2,10,0.5,1.00,0.034206,0.017330,0.021351,0.514600
3,10,1.0,0.50,0.034410,0.017408,0.021448,0.514671
4,10,1.0,0.75,0.032367,0.016497,0.020291,0.513824
5,10,1.0,1.00,0.030519,0.015711,0.019262,0.513051
6,10,1.5,0.50,0.032507,0.016534,0.020344,0.513859
7,10,1.5,0.75,0.030083,0.015402,0.018904,0.512832
8,10,1.5,1.00,0.027844,0.014450,0.017685,0.511958
9,20,0.5,0.50,0.036682,0.018297,0.022688,0.515744


In [17]:
results_df.to_csv('hyper_parameter_training_results_BPR.csv')

## Temp code to retrieve results from text to list

In [5]:
params = []
for f in latent_factors:
    for r in regularization:
        for l in learning_rate:
            params.append([f,r,l])

In [7]:
with open("tmp_BPR_results.txt") as fp:
    text = fp.read()
text

"[\n    {'precision': 0.00510146891416928, 'map': 0.0015033694633245942, 'ndcg': 0.0021315162558968215, 'auc': 0.5016153388481799},\n    {'precision': 0.005467638802051548, 'map': 0.0022934639351212148, 'ndcg': 0.0029528598907335073, 'auc': 0.5021309568635243},\n    {'precision': 0.001188183921903685, 'map': 0.00024903875138810374, 'ndcg': 0.00043161308825532207, 'auc': 0.5003978845943127},\n    {'precision': 0.004481221961225847, 'map': 0.001050715130654907, 'ndcg': 0.0016142223690655615, 'auc': 0.5013023857993317},\n    {'precision': 0.005216052688608629, 'map': 0.0020690025506611494, 'ndcg': 0.002711173039869431, 'auc': 0.50199076293975},\n    {'precision': 0.002959250522477102, 'map': 0.0010683458401969641, 'ndcg': 0.0014697439319069454, 'auc': 0.5011472774375025},\n    {'precision': 0.0015942090356779, 'map': 0.00024641180358203304, 'ndcg': 0.00040603108998945816, 'auc': 0.500253743824585},\n    {'precision': 0.002172109811111139, 'map': 0.0004536812015142101, 'ndcg': 0.0007014983

In [14]:
 metrics = eval(text)

In [15]:
results = []
for p,r in zip(params, metrics):
    results.append(
        (p[0], p[1], p[2], r['precision'], r['map'], r['ndcg'], r['auc'])
    )
results 

[(32,
  0.005,
  0.05,
  0.00510146891416928,
  0.0015033694633245942,
  0.0021315162558968215,
  0.5016153388481799),
 (32,
  0.005,
  0.1,
  0.005467638802051548,
  0.0022934639351212148,
  0.0029528598907335073,
  0.5021309568635243),
 (32,
  0.005,
  0.5,
  0.001188183921903685,
  0.00024903875138810374,
  0.00043161308825532207,
  0.5003978845943127),
 (32,
  0.01,
  0.05,
  0.004481221961225847,
  0.001050715130654907,
  0.0016142223690655615,
  0.5013023857993317),
 (32,
  0.01,
  0.1,
  0.005216052688608629,
  0.0020690025506611494,
  0.002711173039869431,
  0.50199076293975),
 (32,
  0.01,
  0.5,
  0.002959250522477102,
  0.0010683458401969641,
  0.0014697439319069454,
  0.5011472774375025),
 (32,
  0.05,
  0.05,
  0.0015942090356779,
  0.00024641180358203304,
  0.00040603108998945816,
  0.500253743824585),
 (32,
  0.05,
  0.1,
  0.002172109811111139,
  0.0004536812015142101,
  0.0007014983008247475,
  0.5004708951330932),
 (32,
  0.05,
  0.5,
  0.0037414093306065714,
  0.0014